In [28]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from unet.unet_model import UNet
import os
from tqdm import tqdm
import cv2

# Prepare dataloader

In [12]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_filenames = os.listdir(image_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_name = os.path.join(self.mask_dir, self.image_filenames[idx])
        image = cv2.imread(img_name)
        mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        image = image.transpose((2, 0, 1))  # Convert to (C, H, W) format
        image = torch.tensor(image, dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # Convert to (1, H, W) format

        return image, mask

In [13]:
image_dir = "/Users/firmansyahsundana/Documents/research/computer_science/plant-diseases-classification/Data/External/New Plant Diseases Dataset(Augmented)/train"
mask_dir = "/Users/firmansyahsundana/Documents/research/computer_science/plant-diseases-classification/Notebooks/segmentation/results"

In [15]:
dataset = SegmentationDataset(image_dir, mask_dir)

# DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [18]:
dataloader

In [23]:
import torch.optim as optim

model = UNet(n_channels=3, n_classes=2)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [29]:
# Training loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for images, masks in dataloader:
        outputs = model(images)
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

torch.save(model.state_dict(), 'unet_model.pth')

  0%|                                                                                                                                                    | 0/10 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'transpose'